# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846731751394                   -0.70    4.5         
  2   -7.852299820921       -2.25       -1.53    1.0   24.7ms
  3   -7.852609136082       -3.51       -2.56    1.5   26.5ms
  4   -7.852645720220       -4.44       -2.86    2.8   34.6ms
  5   -7.852646427055       -6.15       -3.12    1.0   24.8ms
  6   -7.852646675849       -6.60       -4.13    1.0   25.2ms
  7   -7.852646686664       -7.97       -5.07    2.0    122ms
  8   -7.852646686721      -10.24       -5.28    2.0   30.8ms
  9   -7.852646686728      -11.12       -5.59    1.0   24.9ms
 10   -7.852646686730      -11.88       -6.37    1.0   25.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846701036305                   -0.70           4.2         
  2   -7.852550674994       -2.23       -1.63   0.80    2.5    267ms
  3   -7.852638612223       -4.06       -2.74   0.80    1.0   23.6ms
  4   -7.852646443375       -5.11       -3.33   0.80    1.8   28.1ms
  5   -7.852646678916       -6.63       -4.19   0.80    1.8   27.2ms
  6   -7.852646686449       -8.12       -4.74   0.80    1.8    132ms
  7   -7.852646686708       -9.59       -5.59   0.80    1.5   26.3ms
  8   -7.852646686730      -10.67       -6.30   0.80    2.0   28.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.391754e+01     3.564748e+00
 * time: 0.40828585624694824
     1     1.368204e+00     1.867036e+00
 * time: 0.6065080165863037
     2    -1.424990e+00     2.312771e+00
 * time: 0.6308128833770752
     3    -3.637146e+00     1.873415e+00
 * time: 0.6657350063323975
     4    -5.080181e+00     1.657488e+00
 * time: 0.7008090019226074
     5    -6.756180e+00     8.833501e-01
 * time: 0.7357418537139893
     6    -7.383335e+00     3.442162e-01
 * time: 0.7706279754638672
     7    -7.596196e+00     2.489622e-01
 * time: 0.7950198650360107
     8    -7.683514e+00     1.248396e-01
 * time: 0.8194990158081055
     9    -7.749215e+00     1.287304e-01
 * time: 0.8432350158691406
    10    -7.789909e+00     1.123086e-01
 * time: 0.866908073425293
    11    -7.816821e+00     8.923866e-02
 * time: 0.8909900188446045
    12    -7.833796e+00     6.056390e-02
 * time: 0.9155728816986084
    13    -7.841715e+00     4.587359e-02
 * time: 0.9402298927

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846804017935                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645888728                   -1.64         
  2   -7.852646686730       -6.10       -3.70    1.75s
  3   -7.852646686730      -13.30       -7.24    158ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.242433335164063e-7
|ρ_newton - ρ_scfv| = 2.0366595869600255e-7
|ρ_newton - ρ_dm|   = 2.9468200354043948e-9
